# Advanced Lane Detection & Vehicle Tracking Project

## Import Packages

In [1]:
import glob
import time

import cv2

import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import LinearSVC

from scipy.ndimage.measurements import label

from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline

## Classes

In [2]:
# Class to store camera calibration data
class calibration():
    def __init__(self, obj_pts, img_pts, shape):
        self.ret, self.M, self.dist, self.rvecs, self.tvecs =\
        cv2.calibrateCamera(obj_pts, img_pts, shape, None, None)

# Class to store perspective transform matrices
class transform():
    def __init__(self, src, dst):       
        self.M = cv2.getPerspectiveTransform(src, dst)
        self.Minv = cv2.getPerspectiveTransform(dst, src)
        
# Class to store lane detection data
class lane():
    def __init__(self):
        self.detected = False
        self.x = None
        self.y = None
        self.fit = None
        
# Class to store color and gradient feature extraction parameters
class parameters():
    def __init__(self):
        self.spat_size = (32, 32)
        self.hist_bins = 32
        self.orient = 8
        self.pxs_cell = (8, 8)
        self.cells_block = (2, 2)

# Class to store vehicle detection data
class vehicle():
    def __init__(self):
        self.scaler = None
        self.clf = None
        self.heatmaps = None

## Initiate Class Instances

In [3]:
src = np.float32([[685, 450], [1090, 720], [190, 720], [595, 450]])
dst = np.float32([[990, 0], [990, 720], [290, 720], [290, 0]])
trans = transform(src, dst)

left = lane()
right = lane()

par = parameters()
car = vehicle()

## Lane Detection Functions

In [4]:
# Thresholding function
def threshold(img, thresh):
    binary = np.zeros_like(img)
    binary[(img >= thresh[0]) & (img <= thresh[1])] = 1
    
    return binary

# Thresholded absolute Sobel gradient
def grad_thresh(img, orient, ksize, thresh):
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(img[:, :, 0], cv2.CV_64F, 1, 0, ksize))
    
    elif orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(img[:, :, 0], cv2.CV_64F, 0, 1, ksize))
    
    scaled_sobel = np.uint8(abs_sobel*(255/np.max(abs_sobel)))
    
    return threshold(scaled_sobel, thresh)

# Thresholded magnitude of Sobel gradient
def mag_thresh(img, ksize, thresh):
    sobelx = cv2.Sobel(img[:, :, 0], cv2.CV_64F, 1, 0, ksize)
    sobely = cv2.Sobel(img[:, :, 0], cv2.CV_64F, 0, 1, ksize)
    
    mag_sobelxy = np.sqrt(sobelx**2 + sobely**2)
    
    scaled_sobel = np.uint8(mag_sobelxy*(255/np.max(mag_sobelxy)))

    return threshold(scaled_sobel, thresh)

# Thresholded direction of Sobel gradient
def dir_thresh(img, ksize, thresh):
    abs_sobelx = np.absolute(cv2.Sobel(img[:, :, 0], cv2.CV_64F, 1, 0, ksize))
    abs_sobely = np.absolute(cv2.Sobel(img[:, :, 0], cv2.CV_64F, 0, 1, ksize))
    
    dir_sobelxy = np.arctan2(abs_sobely, abs_sobelx)
    
    return threshold(dir_sobelxy, thresh)

# Warp image perspective
def warp(img, view):
    # Warp image perspective into bird's eye view
    if view == 'b':
        warped = cv2.warpPerspective(img, trans.M, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
        
    # Warp image perspective into driver's view
    elif view == 'd':
        warped = cv2.warpPerspective(img, trans.Minv, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
    
    return warped

# Find initial lane locations
def find_initial(img):
    histogram = np.sum(img[int(img.shape[0]*(3/4)):, :], axis=0) 
    midpoint = histogram.shape[0]//2
    
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint 

    left.x, left.y = find_initial_sub(img, leftx_base)
    right.x, right.y = find_initial_sub(img, rightx_base)
    
    left.fit = np.polyfit(left.y, left.x, 2)
    right.fit = np.polyfit(right.y, right.x, 2)
    
    left.detected = True
    right.detected = True
    
    return None

# Find initial lane locations - sub function
def find_initial_sub(img, xbase):
    num_windows = 9   
    window_height = img.shape[0]//num_windows # 80
    margin = 80
    min_pix = 800
    
    non_zeroy = np.array(img.nonzero()[0])
    non_zerox = np.array(img.nonzero()[1])
    
    lane_inds = []
    shift = 0
    
    # Sliding window technique
    for window in range(num_windows):
        winy_low = img.shape[0] - (window + 1)*window_height # Window lower bound
        winy_high = img.shape[0] - window*window_height # Window upper bound

        winx_left = xbase - margin # Window left bound
        winx_right = xbase + margin # Window right bound

        lane_inds_cur = ((non_zeroy >= winy_low) & (non_zeroy < winy_high) &\
                          (non_zerox >= winx_left) & (non_zerox < winx_right)).nonzero()[0]

        lane_inds.append(lane_inds_cur)
        
        # Center window on average of detected non-zero pixels
        if len(lane_inds_cur) > min_pix:
            xbase_new = np.int(np.mean(non_zerox[lane_inds_cur]))
            shift = xbase_new - xbase
            xbase = xbase_new
        
        # Shift window by previous amount
        else:
            xbase += shift
        

    lane_inds = np.concatenate(lane_inds)

    return non_zerox[lane_inds], non_zeroy[lane_inds]

# Find lane locations using margin around previous lane locations
def find_next(img):
    leftx, lefty = find_next_sub(img, left.fit)
    rightx, righty = find_next_sub(img, right.fit)
    
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Check whether current lanes are within margin of previous lanes
    if np.all([((left_fit[2] > left.fit[2]*0.8) & (left_fit[2] < left.fit[2]*1.2)),\
               ((right_fit[2] > right.fit[2]*0.8) & (right_fit[2] < right.fit[2]*1.2))]):
        
        # Compute weighted average of current and previous polynomial coefficients
        # based on number of non-zero pixels detected
        left_weights = np.array([(len(leftx)/(len(leftx) + len(left.x))),\
                                 (len(left.x)/(len(leftx) + len(left.x)))])

        right_weights = np.array([(len(rightx)/(len(rightx) + len(right.x))),\
                                 (len(right.x)/(len(rightx) + len(right.x)))])

        left.fit = np.average((left_fit, left.fit), axis = 0, weights = left_weights)
        right.fit = np.average((right_fit, right.fit), axis = 0, weights = right_weights)

        left.x, left.y = leftx, lefty
        right.x, right.y = rightx, righty
    
    # If current lanes outside of margin of previous lanes detect lanes from scratch
    else:
        find_initial(img)
    
    return None

def find_next_sub(img, fit):
    margin = 80
    
    non_zeroy = np.array(img.nonzero()[0])
    non_zerox = np.array(img.nonzero()[1])
    
    # Detect non-zero pixels within previously computed polynomial +/- margin
    lane_inds = ((non_zerox > (fit[0]*(non_zeroy**2) + fit[1]*non_zeroy + fit[2] - margin)) &\
                 (non_zerox < (fit[0]*(non_zeroy**2) + fit[1]*non_zeroy + fit[2] + margin)))
    
    return non_zerox[lane_inds], non_zeroy[lane_inds]

# Calculate road curvature radius and vehicle relation to lane center
def info(shape):
    
    # Lane base x-points
    leftx_base = np.mean(left.x[(left.y >= shape[0]*(3/4))])
    rightx_base = np.mean(right.x[(right.y >= shape[0]*(3/4))]) 
    
    # Pixel to meter conversion factors
    xconv = 3.7/(rightx_base - leftx_base)
    yconv = 30/720
    
    # Lane polynomial coefficients
    left_fit = np.polyfit(left.y*yconv, left.x*xconv, 2)
    right_fit = np.polyfit(right.y*yconv, right.x*xconv, 2)
    
    # Average of lane polynomial coefficients
    fit = np.mean((left_fit, right_fit), axis = 0)
    
    # Road curvature radius
    curve_rad = (((1 + (2*fit[0]*shape[0]*yconv + fit[1])**2)**1.5)/np.absolute(2*fit[0]))
    
    # Vehicle relation to lane center
    off_center = (shape[1]/2 - np.mean((leftx_base, rightx_base)))*xconv
    
    return curve_rad, off_center

## Vehicle Detection Functions

In [5]:
# Compute spatial binning feature vector
def get_spat_feat(img):
    return cv2.resize(img, dsize=par.spat_size).ravel() 

# Compute color histogram feature vector
def get_hist_feat(img):
    hist_feat = []
    # Compute histogram for each color channel
    for channel in np.arange(img.shape[2]):
        hist_feat.append(np.histogram(img[:,:,channel], bins=par.hist_bins)[0])
    return np.concatenate(hist_feat)

# Compute HOG feature vector
def get_hog_feat(img, feat_vec):
    hog_feat = []
    # Compute HOG for each color channel
    for channel in np.arange(img.shape[2]):
        hog_feat.append(hog(img[:,:,channel], orientations=par.orient, pixels_per_cell=par.pxs_cell,
                            cells_per_block=par.cells_block, visualise=False, transform_sqrt=True,
                            feature_vector=feat_vec))
    if feat_vec == True:
        return np.concatenate(hog_feat)
    else: return hog_feat

# Extract feature vectors from images
def extract_feat(fnames):
    features = []
    for file in fnames:
        img = mpimg.imread(file)
        # Convert image to YCrCb color space
        img = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        spat_feat = get_spat_feat(img)
        hist_feat = get_hist_feat(img)
        hog_feat = get_hog_feat(img, feat_vec=True)
        # Combine feature vectors
        features.append(np.concatenate((spat_feat, hist_feat, hog_feat)))
    return features

# Find car bounding boxes
def find_cars(img, ystart, ystop, scale):
    # Crop image
    img = img[ystart:ystop,:,:]
    
    # Convert image to YCrCb color space
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    
    # Scale image
    if scale != 1:
        img = cv2.resize(img, (np.int(img.shape[1]/scale), np.int(img.shape[0]/scale)))
    
    # Compute number of HOG cells in x and y directions
    xcells = (img.shape[1]//par.pxs_cell[0])
    ycells = (img.shape[0]//par.pxs_cell[0])
    
    # Training dataset image size
    window = 64
    
    # Compute number of HOG cells and blocks in window
    cells_window = (window//par.pxs_cell[0])
    block_steps = cells_window - par.cells_block[0] + 1
    
    # Compute number of steps in x and y directions
    cells_step = 1
    xsteps = (xcells - cells_window)//cells_step
    ysteps = (ycells - cells_window)//cells_step
    
    hog = get_hog_feat(img, feat_vec = False)
    
    bbox_list = []
    for xb in np.arange(xsteps):
        for yb in np.arange(ysteps):
            # Search window top left coordinate points in cell units
            ypos = yb*cells_step
            xpos = xb*cells_step
            
            # Within search window extract HOG subsample feature vector for each color channel
            hog_feat = []
            for channel in np.arange(img.shape[2]):
                hog_feat.append(hog[channel][ypos:ypos+block_steps, xpos:xpos+block_steps])
            hog_feat = np.concatenate(hog_feat).ravel()
    
            # Search window top left coordinate points in pixel units
            xleft = xpos*par.pxs_cell[0]
            ytop = ypos*par.pxs_cell[0]
            
            # Create search window
            sub_img = cv2.resize(img[ytop:ytop+window, xleft:xleft+window], (window, window))
            
            spat_feat = get_spat_feat(sub_img)
            hist_feat = get_hist_feat(sub_img)
            
            # Combine and scale feature vectors
            features = np.concatenate((spat_feat, hist_feat, hog_feat))
            scaled_features = car.scaler.transform(features)
            
            # Predict class and save confidence
            confidence = car.clf.decision_function(scaled_features)
            
            # Save search window coordinate points
            if confidence > 0.5:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                bbox_list.append(((xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart))) 
    return bbox_list

# Create heatmap from bounding boxes
def create_heatmap(heatmap, bbox_list):
    for bbox in bbox_list:
        heatmap[bbox[0][1]:bbox[1][1], bbox[0][0]:bbox[1][0]] += 1
    return heatmap

## Draw Functions

In [6]:
# Draw identified lane and road information on image
def draw_lane(img):
    # Create blank canvas to draw lane on
    blank = np.zeros_like(img).astype(np.uint8)
    
    # Create y-points
    ploty = np.linspace(0, blank.shape[0] - 1, blank.shape[0])
    
    # Compute x-points
    leftx = left.fit[0]*ploty**2 + left.fit[1]*ploty + left.fit[2]
    rightx = right.fit[0]*ploty**2 + right.fit[1]*ploty + right.fit[2]
    
    # Combine lane points
    left_pts = np.array([np.transpose(np.vstack([leftx, ploty]))])
    right_pts = np.array([np.flipud(np.transpose(np.vstack([rightx, ploty])))])
    pts = np.hstack((left_pts, right_pts))
    
    # Draw lane
    lane = cv2.fillPoly(blank, np.int_([pts]), (0, 255, 0))
    
    # Project lane onto road
    projected = warp(lane, view = 'd')
    
    # Combine image with projected lane
    return cv2.addWeighted(img, 1, projected, 0.3, 0)
    
# Draw road curvature radius and vehicle relation to lane center on image
def draw_info(img):
    curve_rad, off_center = info(img.shape)
    
    curve_text = 'Curvature radius = {:d}m'.format(int(curve_rad))
    
    if off_center < 0:
        off_text = 'Vehicle {:.2f}m left of center'.format(abs(off_center))
    
    elif off_center > 0:
        off_text = 'Vehicle {:.2f}m right of center'.format(abs(off_center))

    # Draw curvature radius on image
    cv2.putText(img, curve_text, org = (10, 30), fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                fontScale = 1, thickness = 2, color = (0, 255, 0), bottomLeftOrigin = False)
    
    # Draw vehicle relation to lane center on image
    return cv2.putText(img, off_text, org = (10, 60), fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                       fontScale = 1, thickness = 2, color = (0, 255, 0), bottomLeftOrigin = False)

def draw_bbox(img, cars):
    # Draw bounding box for each car
    for car in np.arange(1, cars[1] + 1):
        non_zerox = (cars[0] == car).nonzero()[1]
        non_zeroy = (cars[0] == car).nonzero()[0]
        bbox = (np.min(non_zerox), np.min(non_zeroy)), (np.max(non_zerox), np.max(non_zeroy))
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,256), 4)
        
    return img

## Calibrate Camera

In [7]:
# Load calibration image filenames
calibration_fnames = glob.glob('camera_cal/calibration*.jpg')

numx = 9 # Number of inner row corners
numy = 6 # Number of inner column corners

cal_obj_pts = []
cal_img_pts = []

# Create array of known object points
obj_pts = np.zeros((numx*numy, 3), np.float32)
obj_pts[:, :2] = np.mgrid[0:numx, 0:numy].T.reshape(-1, 2)

# Loop over calibration images
for fname in calibration_fnames:
    img = mpimg.imread(fname) # Load image
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # Convert image to grayscale
    ret, img_pts = cv2.findChessboardCorners(gray, (numx, numy), None) # Detect image points
    
    # If chessboard corners detected, append object and image points to those previously detected
    if ret == True: 
        cal_obj_pts.append(obj_pts)
        cal_img_pts.append(img_pts)

# Create an instance of the camera calibration data class
cal = calibration(cal_obj_pts, cal_img_pts, img.shape[0:2])

## Extract Features

In [8]:
# Load car and non-car image filenames
car_fnames = glob.glob('training_data/vehicles/vehicle*.jpg')
non_car_fnames = glob.glob('training_data/non_vehicles/non_vehicle*.jpg')

print('Extracting features...')

# Extract color and gradient features from car and non-car images
car_feat = extract_feat(car_fnames)
non_car_feat = extract_feat(non_car_fnames)

# Combine car and non-car features
X = np.vstack((car_feat, non_car_feat)).astype(np.float64)

print('Standardizing features...')

# Standardize features by zero mean centering and scaling to standard deviation
car.scaler = StandardScaler().fit(X)
scaled_X = car.scaler.transform(X)

# Create labels for car and non-car features
y = np.hstack((np.ones(len(car_feat), dtype = np.int8), np.zeros(len(non_car_feat), dtype = np.int8)))

print('Creating training and test subsets...')

# Randomly split features and labels into training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2,
                                                    random_state=np.random.randint(0, 100))

print('Number of training examples = ', len(X_train))
print('Number of test examples = ', len(X_test))

Extracting features...
Standardizing features...
Creating training and test subsets...
Number of training examples =  14208
Number of test examples =  3552


## Train Classifier

In [9]:
# Initiate Linear SVC classifier
car.clf = LinearSVC()

print('Training...')

start = time.time()
car.clf.fit(X_train, y_train)
stop = time.time()

print('Training Time = {:.3f} s'.format(stop - start))
print('Training Accuracy = {:.3f}'.format(car.clf.score(X_train, y_train)))
print('Test Accuracy = {:.3f}'.format(car.clf.score(X_test, y_test)))

Training...
Training Time = 32.683 s
Training Accuracy = 1.000
Test Accuracy = 0.993


## Lane Detection Pipeline

In [10]:
def lane_pipeline(img):    
    # Threshold image
    binary_gradx = grad_thresh(img, orient = 'x', ksize = 3, thresh = (20, 255))
    binary_grady = grad_thresh(img, orient = 'y', ksize = 3, thresh = (20, 255))
    
    binary_grad_mag = mag_thresh(img, ksize = 3, thresh = (40, 255))
    binary_grad_dir = dir_thresh(img, ksize = 3, thresh = (0.7, 1.3))
    
    binary_rch = threshold(img[:, :, 0], thresh = (140, 255))
    binary_sch = threshold(cv2.cvtColor(img, cv2.COLOR_RGB2HLS)[:, :, 2],\
                           thresh = (100, 255))
    
    # Combine binary images
    combined = np.zeros_like(binary_gradx)
    combined[((binary_gradx == 1) & (binary_grady == 1)) |\
                ((binary_grad_mag == 1) & (binary_grad_dir == 1)) |\
                ((binary_rch == 1) & (binary_sch == 1))] = 1
    
    # Warp image perspective into bird's eye view
    warped = warp(combined, view = 'b')
    
    # Find lane locations from scratch
    if left.detected == False & right.detected == False:
        find_initial(warped)
    
    # Find lane locations using margin around previously found lane locations
    elif left.detected == True & right.detected == True:
        find_next(warped)
   
    return None

## Vehicle Detection Pipeline

In [11]:
def vehicle_pipeline(img):
    # Parameters
    ystart_list = [400, 400]
    ystop_list = [480, 528]
    scale_list = [1.25, 1.5]
    thresh = 4
    frames = 4
    
    # Find car bounding boxes
    bbox_list = [] 
    for ystart, ystop, scale in zip(ystart_list, ystop_list, scale_list):
        bbox = find_cars(img, ystart, ystop, scale)
        if len(bbox) > 0:
            bbox_list.append(bbox)

    # Create a heatmap if bounding boxes were found
    if len(bbox_list) > 0:
        bbox_list = np.concatenate(bbox_list)
        car.heatmaps.append(create_heatmap(np.zeros_like(img[:,:,0]), bbox_list))
    
    # Combine last n stored heatmaps
    heatmap = np.zeros_like(img[:,:,0])
    for i in range(len(car.heatmaps)):
        heatmap += car.heatmaps[i]
    
    # Delete first in n stored heatmaps
    if len(car.heatmaps) > frames:
        del car.heatmaps[0]
    
    # Threshold heatmap
    heatmap[heatmap < thresh] = 0

    # Label individual heatmap regions
    return label(heatmap)

## Combined Pipeline

In [12]:
def pipeline(img):
    
    # Undistort image using previously computed camera calibration matrix
    img = cv2.undistort(img, cal.M, cal.dist, None, cal.M)
    
    # Detect lane
    lane_pipeline(img)
    
    # Detect vehicles
    cars = vehicle_pipeline(img)
    
    # Draw lane, info, and bounding boxes
    lane = draw_lane(img)
    info = draw_info(lane)    
    return draw_bbox(info, cars)

## Output

In [14]:
car.heatmaps = []
video_output = 'videos/output/project_video.mp4'
video_input = VideoFileClip('videos/input/project_video.mp4')
video_clip = video_input.fl_image(pipeline)
%time video_clip.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video videos/output/project_video.mp4
[MoviePy] Writing video videos/output/project_video.mp4



100%|█████████▉| 1260/1261 [12:36<00:00,  1.69it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: videos/output/project_video.mp4 

CPU times: user 18min 38s, sys: 1min 36s, total: 20min 14s
Wall time: 12min 38s


In [15]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))